In [1]:
# Import libraries

import xarray as xr
import numpy as np
import rioxarray
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import oggm
from oggm import cfg, utils, workflow, tasks, graphics
from oggm import entity_task, global_tasks
from oggm.utils import compile_climate_input
from oggm.core import gis
from oggm.utils import DEM_SOURCES
from pathlib import Path
import os
import logging
from datetime import datetime, timedelta

pd.set_option('display.max_columns',None)

cfg.initialize(logging_level='WARNING')
cfg.PARAMS['border'] = 10
cfg.PARAMS['use_multiprocessing'] = True 
cfg.PARAMS['continue_on_error'] = True
# Module logger
log = logging.getLogger('.'.join(__name__.split('.')[:-1]))


2023-04-18 13:26:08: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2023-04-18 13:26:08: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2023-04-18 13:26:08: oggm.cfg: Multiprocessing: using all available processors (N=4)
2023-04-18 13:26:09: oggm.cfg: PARAMS['border'] changed from `80` to `10`.
2023-04-18 13:26:09: oggm.cfg: Multiprocessing switched ON after user settings.
2023-04-18 13:26:09: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.


In [2]:
# Workspace path is path to store OGGM data
parent_path = os.path.dirname(Path().resolve())
workspace_path = os.path.join(parent_path, 'shared-public/MLinGlaciology/BYOP/data/point_mb_glaciers_norway/oggm_data/')

cfg.PATHS['working_dir'] = workspace_path

In [3]:
# Convert datetimes to right format
def decimal_to_datetime(ds):
    datetimes = []
    for dec_year in ds.time.data:
        year = int(dec_year)
        rem = dec_year - year

        base = datetime(year, 1, 1)
        calendar_time = base + timedelta(seconds=(base.replace(year=base.year + 1) - base).total_seconds() * rem)
        datetimes.append(calendar_time)
        
    climate_ds['time'] = datetimes
        
    return climate_ds

In [4]:
@entity_task(log)
def get_gridded_features(gdir):
    """Retrieves and processes gridded and climate data for a gdir in order to create the dataset
        to be fed to machine learning models.
    """
    # Retrieve gridded data
    with xr.open_dataset(gdir.get_filepath('gridded_data')) as ds:
        gridded_ds = ds.load()
    # Retrieve climate data
    with xr.open_dataset(gdir.get_filepath('climate_historical')) as ds:
        climate_ds = ds.load()
    
    #### Climate data ####
    # First we start by processing the climate data
    # Trim climate dataset to desired period of the Hugonnet et al. (2021) dataset
    climate_ds = climate_ds.sel(time=slice('2000-01-01','2019-12-01')) 
    
    # Now we downscale the climate data to the specific glacier 
    # Temperature
    temps_2D = np.empty((climate_ds.temp.data.size, gridded_ds.topo.data.shape[0], gridded_ds.topo.data.shape[1]))
    i = 0
    for temp in climate_ds.temp.data:
        temps_2D[i,:,:] = np.tile(temp,gridded_ds.topo.data.shape) + 6.0/1000.0*(gridded_ds.topo.data - climate_ds.ref_hgt.data)
        i=i+1
    
    PDD_2D = np.sum(np.where(temps_2D > 0.0, temps_2D, 0.0), axis=0)
    
    # Rain
    rain_period_2D = np.empty((climate_ds.prcp.data.size, gridded_ds.topo.data.shape[0], gridded_ds.topo.data.shape[1]))
    i = 0
    for prcp in climate_ds.prcp.data:
        rain_period_2D[i,:,:] = np.tile(prcp,gridded_ds.topo.data.shape) 
        i=i+1
    
    rain_2D = np.sum(rain_period_2D, axis=0)
    
        # Snow
    snow_2D = np.where(temps_2D < 0.0, rain_period_2D, 0.0)
        
    # Now we create a dictionary with the full dataset, including the previous climate data
    # and all the interesting gridded datasets 
    training_data = {'PDD_2D': PDD_2D,
                 'rain_2D': rain_2D,
                 'snow_2D': snow_2D,
                 'topo': gridded_ds.topo.data,
                 'aspect': gridded_ds.aspect.data,
                 'slope': gridded_ds.slope.data,
                 'dis_from_border': gridded_ds.dis_from_border.data,
                 'glacier_mask': gridded_ds.glacier_mask.data,
                 'millan_ice_thickness': gridded_ds.millan_ice_thickness.data,
                 'hugonnet_dhdt': gridded_ds.hugonnet_dhdt.data,
                 'ID': gdir.rgi_id
        }
    
    return training_data

In [5]:
rgi_region = '08' # Scandinavia
rgi_version = '6'
rgi_dir = utils.get_rgi_dir(version=rgi_version)
path = utils.get_rgi_region_file(region=rgi_region, version=rgi_version)
rgidf = gpd.read_file(path)
# We use the directories with the shop data in it: "W5E5_w_data"
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/elev_bands/W5E5_w_data/'
gdirs = workflow.init_glacier_directories(rgidf, from_prepro_level=3, prepro_base_url=base_url, prepro_border=10, reset=True, force=True)

2023-04-18 13:27:30: oggm.workflow: init_glacier_directories from prepro level 3 on 3417 glaciers.
2023-04-18 13:27:30: oggm.workflow: Execute entity tasks [gdir_from_prepro] on 3417 glaciers


In [6]:
# Tested tasks
task_list = [
    tasks.gridded_attributes,
    tasks.gridded_mb_attributes,
]
for task in task_list:
    workflow.execute_entity_task(task, gdirs, print_log=False)

2023-04-18 13:39:00: oggm.workflow: Execute entity tasks [gridded_attributes] on 3417 glaciers
2023-04-18 13:42:51: oggm.workflow: Execute entity tasks [gridded_mb_attributes] on 3417 glaciers


In [7]:
for gdir in gdirs:
    if (gdir.name == 'Nigardsbreen'):
        break

In [8]:
with xr.open_dataset(gdir.get_filepath('gridded_data')) as ds:
    ds = ds.load()
# List all variables
ds

<xarray.Dataset>
Dimensions:                  (x: 108, y: 108)
Coordinates:
  * x                        (x) float32 -4.117e+03 -4.016e+03 ... 6.69e+03
  * y                        (y) float32 6.848e+06 6.848e+06 ... 6.837e+06
Data variables: (12/20)
    topo                     (y, x) float32 545.2 473.4 365.1 ... 716.0 623.4
    topo_smoothed            (y, x) float32 585.5 530.5 436.3 ... 667.4 612.1
    topo_valid_mask          (y, x) int8 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    glacier_mask             (y, x) int8 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    glacier_ext              (y, x) int8 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    consensus_ice_thickness  (y, x) float32 nan nan nan nan ... nan nan nan nan
    ...                       ...
    aspect                   (y, x) float32 0.528 0.638 0.7656 ... 0.7515 0.706
    slope_factor             (y, x) float32 1.123 1.071 1.016 ... 1.192 1.238
    dis_from_border          (y, x) float32 9.213e+03 9.143e+03 ... 1.428e+03
    catchment_area           (y, x) float32 nan nan nan nan ... nan nan nan nan
    lin_mb_above_z           (y, x) float32 nan nan nan nan ... nan nan nan nan
    oggm_mb_above_z          (y, x) float32 nan nan nan nan ... nan nan nan nan
Attributes:
    author:         OGGM
    author_info:    Open Global Glacier Model
    pyproj_srs:     +proj=tmerc +lat_0=0 +lon_0=7.099 +k=0.9996 +x_0=0 +y_0=0...
    max_h_dem:      1970.0707
    min_h_dem:      88.412544
    max_h_glacier:  1939.6395
    min_h_glacier:  311.37354